<a href="https://colab.research.google.com/github/Siddharth02003/CS6910_assignments/blob/main/CS6910_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import copy 
import random
import tensorflow as tf
from tqdm import tqdm


In [2]:
!pip install wandb -qqq
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.5 MB/s eta 0:00:00


In [3]:
def load_dataset():
    (X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
    return {
            'X_train': X_train,
            'Y_train': Y_train,
            'X_test': X_test,
            'Y_test': Y_test
        }

In [4]:
class_labels={0:'T-shirt',1:'Trouser',2:'Pullover',3:'Dress',4:'Coat',5:'Sandal',6:'Shirt',7:'Sneaker',8:'Bag',9:'Ankle_Boot'}

def label_to_name(label):
  if enumerate(label):
    l_names=[]
    for l in label:
      l_names.append(class_labels[l])
    return l_names
  else:
    return class_labels[label]

In [5]:
!wandb login --relogin
entity_name="siddharth-s"

project_name="FODL_Assignment_1"

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
def log_images():
  images=[]
  labels=[]
  dataset=load_dataset()
  X_train=dataset['X_train']
  y_train=dataset['Y_train']
  wandb.init(entity=entity_name,project=project_name, name="log_images")
  for i in range(100):
    if len(labels)==10:
      break
    if class_labels[y_train[i]] not in labels:
      images.append(X_train[i])
      labels.append(class_labels[y_train[i]])
  wandb.log({"Images": [wandb.Image(img, caption=caption) for img, caption in zip(images,labels)]})

In [7]:
class activation(): 
  def __init__(self,a):
    self.a=a

  def sigmoid(self,a):
    try:
      return (1.0/(1.0+np.exp(-a)))
    except:
      print("error")

  def relu(self,a):
    return (np.maximum(0,a))

  def tanh(self,a):
    return np.tanh(a)

  def softmax(self,a):
    try:
      return(np.exp(a)/np.sum(np.exp(a)))
    except:
      print("error")

  def sigmoid_derivative(self,x):
    return self.sigmoid(x)*(1-self.sigmoid(x))

  def tanh_derivative(self,x):
    return 1.0 -self.tanh(x)**2

  def relu_derivative(self,x):
    return 1. * (x>0)
     
  def softmax_derivative(self,x):
    return self.softmax(x) * (1-self.softmax(x))

  def derivative(self,x,activation):
    if activation == "sigmoid":
      return self.sigmoid_derivative(x)
    elif activation == "tanh":
      return self.tanh_derivative(x)
    elif activation == "relu":
      return self.relu_derivative(x)

In [8]:
class weights():
  def __init__(self,layers):
    self.layers=layers

  def Xavier(self,layers):
    params = {}
    for i in range(1,len(layers)):
       norm_xav=np.sqrt(6)/np.sqrt(layers[i]+layers[i-1])
       params["w"+str(i)]=np.random.randn(layers[i],layers[i-1])*norm_xav
       params["b"+str(i)]=np.zeros((layers[i],1))
    return params

  def Random(self,layers):
    params = {}
    for i in range(1,len(layers)):
       params["w"+str(i)]=0.01*np.random.randn(layers[i],layers[i-1])
       params["b"+str(i)]=0.01*np.random.randn(layers[i],1)
    return params

  def weight_init(self,init_type = "random"):
    params={}
    if(init_type=="xavier"):
      params = self.Xavier(self.layers)
    elif(init_type=="random"):
      params = self.Random(self.layers)
    else:
      print("invalid activation function")
    return params

In [9]:
def squared_loss(y, y_hat):
  error = np.sum(((y - y_hat)**2) / (2 * len(y)))
  return error
def CrossEntropy(y, y_hat):
  error = - np.sum( np.multiply(y , np.log(y_hat)))/len(y)
  return error
def loss_calc(loss_name, y, y_hat, lambd, layers, parameters):
  error=0
  if(loss_name == "squared_loss"):
    error=squared_loss(y, y_hat)
  elif(loss_name == "cross_entropy"):
    error= CrossEntropy(y, y_hat)
  regularized_error = 0.0
  for i in range(len(layers)-1, 0, -1):
    regularized_error += (np.sum(parameters["w"+str(i)]))**2
  regularized_error = error + ((lambd/(2*len(y)))*(regularized_error))
  return regularized_error




In [10]:
class network():
  def __init__(self,X,y,params,active,layers,loss_type):
    self.X=X
    self.y=y
    self.params=params
    self.active=active
    self.layers=layers
    self.loss_type=loss_type

  def forward_prop(self):
   out=copy.deepcopy(self.X)
   out=out.reshape(-1,1)
   h=[out]
   a=[out] 

   act=activation(a)

   if(self.active=="sigmoid"):
     for i in range(1,len(self.layers)-1):
       weights = self.params["w"+str(i)]
       biases = self.params["b"+str(i)]

       out = np.dot(weights,h[i-1])+biases
       a.append(out)
       post_a = act.sigmoid(out)
       h.append(post_a)
  
   elif(self.active=="tanh"):
     for i in range(1,len(self.layers)-1):
       weights=self.params["w"+str(i)]
       biases=self.params["b"+str(i)]
      
       out=np.dot(weights,h[i-1])+biases
       a.append(out)
       post_a=act.tanh(out)
       h.append(post_a)
  
   elif(self.active=="relu"):
     for i in range(1,len(self.layers)-1):
       weights=self.params["w"+str(i)]
       biases=self.params["b"+str(i)]
      
       out=np.dot(weights,h[i-1])+biases
       a.append(out)
       post_a=act.relu(out)
       h.append(post_a)       

   else:
     print("Enter a valid activation function") 
   weights=self.params["w"+str(len(self.layers)-1)]
   biases=self.params["b"+str(len(self.layers)-1)]
  
   out=np.dot(weights,h[len(self.layers)-2])+biases
   a.append(out)
   y_hat=act.softmax(out)
   h.append(y_hat)
   return h,a,y_hat

  def backward_prop(self,y,y_hat,h,a,params,layers):
    grad = {}
    act=activation(self.active)
    if self.loss_type == "squared_loss":
      grad["dh"+str(len(layers)-1)] = (y_hat - y)
      grad["da"+str(len(layers)-1)] = (y_hat - y) * act.softmax_derivative(a[len(layers)-1])

    elif self.loss_type == 'cross_entropy':
      grad["da"+str(len(layers)-1)] = -(y-y_hat)
      grad["dh"+str(len(layers)-1)] = -(y/y_hat)

    for i in range(len(layers)-1, 0, -1 ):
      grad["dw"+str(i)] = np.dot(grad["da"+str(i)], np.transpose(h[i-1]))
      grad["db"+str(i)] = grad["da"+str(i)]
      if i > 1:
        grad["dh"+str(i-1)] = np.dot(np.transpose(params["w"+str(i)]), grad["da"+str(i)])
        grad["da"+str(i-1)] = np.multiply(grad["dh" + str(i-1)], act.derivative(a[i-1],self.active))
    return grad

In [11]:
def accuracy_calc(res,y_t):
    acc=0.0   
    for x in range(len(res)):
      if(res[x].argmax()==y_t[x].argmax()):
        acc+=1
    acc=acc/len(y_t)
    return(acc*100)

In [12]:
def run_inference(X,y,parameters,activation,layers):
    result = []
    for i in range(len(X)):
      nn=network(X[i], y[i], parameters, activation, layers,"squared_loss")
      h,a,y_hat = nn.forward_prop()
      y_hat = y_hat.flatten()
      result.append(y_hat)
    return result

In [13]:
def calculate_grad(X, Y, parameters, activation, layers, loss_function):
  grads={}
  grads.clear() 
  for j in range(len(X)):
    y = np.reshape(Y[j], (-1,1))

    nn=network(X[j], y, parameters, activation, layers, loss_function)
    h,a,y_hat = nn.forward_prop()
    new_grads = nn.backward_prop(y,y_hat,h,a,parameters,layers)

    if j == 0:
      grads = copy.deepcopy(new_grads)
    else:
      for k in range(len(layers)-1,0,-1):
        grads["dw"+str(k)] += new_grads["dw"+str(k)]
        grads["db"+str(k)] += new_grads["db"+str(k)]
  return grads

In [14]:
def gradient_descent(X_train, y_train, eta, max_epochs, layers, mini_batch_size, lambd,loss_function, activation, parameters,optimiser,wandb_log=False):
  grads={}
  train_loss = []
  val_loss = []
  train_acc = []
  val_acc = []

  for t in tqdm(range(max_epochs)):
    for i in range(0, len(X_train), mini_batch_size):
      
      grads.clear()

      if str(optimiser) == "nesterovacc_gd":
        opt=optimiser(grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters)
        param_lookahead,update_history=opt.paramlookahead()

      X = X_train[i:i + mini_batch_size]
      Y = y_train[i:i + mini_batch_size]
      
      if str(optimiser) == "nesterovacc_gd":
        grads = calculate_grad(X,Y,param_lookahead,activation,layers,loss_function)
      else: 
        grads = calculate_grad(X,Y,parameters,activation,layers,loss_function)

      opt=optimiser(grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i)
      parameters=opt.get_params()
    
    #Calculating train loss and accuracies
    res = run_inference(X_train,y_train,parameters, activation, layers)
    train_err = loss_calc(loss_function,y_train,res,lambd,layers,parameters) 
    train_ac=accuracy_calc(res, y_train)
    train_loss.append(train_err)
    train_acc.append(train_ac)

    #Calculating validation loss
    res = run_inference(X_val, y_val, parameters, activation, layers)
    val_err = loss_calc(loss_function, y_val, res, lambd, layers, parameters )
    val_ac=accuracy_calc(res, y_val)
    val_loss.append(val_err)
    val_acc.append(val_ac)

    if(wandb_log==True):
      log_dict = {"Train_Accuracy": train_ac, "Validation_Accuracy": val_ac, \
                  "Train_Loss": train_err, "Validation_loss": val_err, "epoch": t}
                  
      wandb.log(log_dict)

  return parameters, train_acc, val_acc

In [15]:
class stochastic_gd():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i
    
  def get_params(self):
    for j in range(len(self.layers)-1,0,-1):
        self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - (self.eta * self.grads["dw"+str(j)])
        self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - (self.eta * self.grads["db"+str(j)])
    return self.parameters

In [16]:
class momentum_gd():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i
    
  def get_params(self):
    update_history={}
    if self.i == 0 :
        for j in range(len(self.layers)-1, 0, -1):
          update_history["w"+str(j)] = self.eta*self.grads["dw"+str(j)]
          update_history["b"+str(j)] = self.eta*self.grads["db"+str(j)]
    else:
        for j in range(len(self.layers)-1, 0, -1):
          update_history["w"+str(j)] = (self.gamma*update_history["w"+str(j)]) + (self.eta*self.grads["dw"+str(j)])
          update_history["b"+str(j)] = (self.gamma*update_history["b"+str(j)]) + (self.eta*self.grads["db"+str(j)])
    for j in range(len(self.layers)-1,0,-1):
        self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - update_history["w"+str(j)]
        self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - update_history["b"+str(j)]
    return self.parameters

In [17]:
class nesterovacc_gd():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i

  def paramlookahead(self):
    update_history={}
    if self.i==0:
        param_lookahead = copy.deepcopy(self.parameters)
    else:
        for j in range(len(self.layers)-1, 0, -1):
          param_lookahead['w'+str(j)] = self.parameters['w'+str(j)] + (self.gamma*update_history["w"+str(j)])
    return param_lookahead,update_history

  def get_params(self,update_history):
    param_lookahead,update_history=self.paramlookahead()
    if self.i == 0 :
        for j in range(len(self.layers)-1, 0, -1):
          update_history["w"+str(j)] = self.eta*self.grads["dw"+str(j)]
          update_history["b"+str(j)] = self.eta*self.grads["db"+str(j)]
    else:
        for j in range(len(self.layers)-1, 0, -1):
          update_history["w"+str(j)] = (self.gamma*update_history["w"+str(j)]) + (self.eta*self.grads["dw"+str(j)])
          update_history["b"+str(j)] = (self.gamma*update_history["b"+str(j)]) + (self.eta*self.grads["db"+str(j)])
    for j in range(len(self.layers)-1,0,-1):
        self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - update_history["w"+str(j)]
        self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - update_history["b"+str(j)]
    return self.parameters


In [1]:
class rmsprop():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i
    self.beta = 0.9 
    self.epsilon=1e-8
  
  def momenta(self):
    update_history={}
    v={}
    for i in range(len(self.layers)-1,0,-1):
      update_history["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      update_history["b"+str(i)]=np.zeros((self.layers[i],1))
    for i in range(len(self.layers)-1,0,-1):
      v["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      v["b"+str(i)]=np.zeros((self.layers[i],1))
    return v,update_history
     
  def get_params(self):
    v,update_history=self.momenta()
    for iq in range(len(self.layers)-1,0,-1):
        v["w"+str(iq)]=self.beta*v["w"+str(iq)]+(1-self.beta)*self.grads["dw"+str(iq)]**2
        v["b"+str(iq)]=self.beta*v["b"+str(iq)]+(1-self.beta)*self.grads["db"+str(iq)]**2     
        update_history["w"+str(iq)]=self.eta*np.multiply(np.reciprocal(np.sqrt(v["w"+str(iq)]+self.epsilon)),self.grads["dw"+str(iq)])
        update_history["b"+str(iq)]=self.eta*np.multiply(np.reciprocal(np.sqrt(v["b"+str(iq)]+self.epsilon)),self.grads["db"+str(iq)])
    for j in range(len(self.layers)-1,0,-1):
        self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - update_history["w"+str(j)]
        self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - update_history["b"+str(j)]
    return self.parameters

Preparing Dataset

In [18]:
from sklearn.model_selection import train_test_split
(train_x,train_y),(test_x,test_y)=fashion_mnist.load_data()
num_classes = 10
labels=['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

print("Number of data points in train data (initially) - ", len(train_x))
print("Number of data points in test data (initially) - ", len(test_x))


#performing the train-validation split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=40)
  

print("Shape of each image - 28x28" )
image_shape=train_x.shape[1]*train_x.shape[2]
print("shape of each image (1D) - ",image_shape)
  
#storing the number of points in each set
train_image_count=len(train_x)
val_image_count = len(val_x)
test_image_count=len(test_x)
  
X_train=np.zeros((train_image_count,image_shape))
X_val=np.zeros((val_image_count,image_shape))
X_test=np.zeros((test_image_count,image_shape))
  
# converting the images into grayscale by normalizing
for i in range(train_image_count):
  X_train[i]=(copy.deepcopy(train_x[i].flatten()))/255.0 
for i in range(val_image_count):
  X_val[i]=(copy.deepcopy(val_x[i].flatten()))/255.0
for i in range(test_image_count):
  X_test[i]=(copy.deepcopy(test_x[i].flatten()))/255.0
  
y_train = np.zeros((train_y.size, 10))
y_train[np.arange(train_y.size), train_y] = 1

y_val = np.zeros((val_y.size, 10))
y_val[np.arange(val_y.size), val_y] = 1

y_test = np.zeros((test_y.size, 10))
y_test[np.arange(test_y.size), test_y] = 1


4422102/4422102 [==============================] - 0s 0us/step
Number of data points in train data (initially) -  60000
Number of data points in test data (initially) -  10000
Shape of each image - 28x28
shape of each image (1D) -  784


In [19]:
def fit(X_train, y_train, layers,wandb_log, learning_rate = 0.0001, initialization_type = "random", activation_function = "sigmoid", loss_function = "cross_entropy", mini_batch_Size = 32, max_epochs = 5, lambd = 0,optimization_function = stochastic_gd): 

  w=weights(layers)
  parameters = w.weight_init(init_type = initialization_type)
  parameters, train_acc, val_acc = gradient_descent(X_train, y_train,learning_rate, max_epochs, layers, mini_batch_Size, lambd, loss_function, activation_function, parameters,optimization_function,wandb_log)
  
  print("Training Accuracy:",train_acc[-1])
  print("Validation Accuracy:",val_acc[-1])

  return parameters


In [20]:
# layers=[784,32,10]
# wandb.init(entity_name,project_name)
# wandb_log=True
# fit(X_train, y_train, layers,wandb_log)

In [21]:
def train():

  #Declaring the dictionary with the default hyperparameters
  config_defaults = {
      'number_hidden_layers': 2,
      'number_neurons': 32,
      'learning_rate': 0.001,
      'initialization_type': "xavier",
      'activation_function':'sigmoid',
      'mini_batch_size' : 64,
      'max_epochs': 5,
      'lambd': 0,
      'optimization_function': "adam"
  }

  # Initializing the wandb run
  wandb.init(config=config_defaults)
  config = wandb.config


  # Constructing the layers i.e., the architecture of our neural network
  layers = [784]
  for i in range(config.number_hidden_layers):
    layers = layers + [config.number_neurons]
  layers  = layers + [10]

  #Collecting all the hyperparameters from the wandb run
  learning_rate = config.learning_rate
  initialization_type = config.initialization_type
  activation_function = config.activation_function
  loss_function = "cross_entropy"
  mini_batch_size = config.mini_batch_size
  max_epochs = config.max_epochs
  lambd = config.lambd
  opt_fun = config.optimization_function
  hidden_layers=config.number_hidden_layers

  #Calling the respective hyperparameters
  if opt_fun == "adam":
    optimization_function = adam
  elif opt_fun == "nadam":
    optimization_function = nadam
  elif opt_fun == "stochastic_gd":
    optimization_function = stochastic_gd
  elif opt_fun == "momentum_gd":
    optimization_function = momentum_gd
  elif opt_fun == "nesterov-acc_gd":
    optimization_function = nesterovacc_gd
  elif opt_fun == "rmsprop":
    optimization_function = rmsprop
  else:
    print("Wrong optimization function")
    exit()


  #Forming meaningful run name using the hyperparameters
  name_run = str(hidden_layers) + "_" + initialization_type[0] + "_" + \
  activation_function[0] + "_" + str(learning_rate) 
  "_" + opt_fun[:4]

  wandb.run.name = name_run
  wandb_log=True
  
  parameters = fit(X_train, y_train, layers,wandb_log, learning_rate, initialization_type, activation_function, loss_function, mini_batch_size, 5,0,optimization_function)
  
  wandb.run.save()
  wandb.run.finish()

In [22]:
def do_sweep(entity_name,project_name):

  #Using bayes method for hyperparameter sweeps to curb the unnecessary configurations
  hyperparameters = {
      "learning_rate":{
        'values': [0.001, 0.0001]
      },

      "number_hidden_layers": {
          'values' : [3, 4, 5]
      },

      "number_neurons": {
        'values': [32, 64, 128]
      },

      "initialization_type": {
          'values' : ["xavier", "random"]
      },

      "activation_function": {
          'values': ["sigmoid", "tanh", "relu"]
      },

      "mini_batch_size": {
          'values': [16,32,64,128]
      },

      "max_epochs": {
          'values': [5, 10, 20]
      },

      "lambd": {
          'values': [0, 0.0005, 0.5]
      },

      "optimization_function": {
          'values': ["stochastic_gd","momentum_gd","nesterovacc_gd"]
      }

  }


  #Using bayes method for hyperparameter sweeps to curb the unnecessary configurations
  sweep_config = {
      'method' : 'bayes',
      'metric' :{
          'name': 'Validation_Accuracy',
          'goal': 'maximize'
      },
      'parameters': hyperparameters
  }

  sweep_id = wandb.sweep(sweep_config, entity=entity_name, project=project_name)
  wandb.agent(sweep_id, train)

In [23]:
do_sweep(entity_name,project_name)

Create sweep with ID: tbugiiwc
Sweep URL: https://wandb.ai/siddharth-s/FODL_Assignment_1/sweeps/tbugiiwc


wandb: Agent Starting Run: ymz1h82u with config:
wandb: 	activation_function: relu
wandb: 	initialization_type: random
wandb: 	lambd: 0.5
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 20
wandb: 	mini_batch_size: 128
wandb: 	number_hidden_layers: 5
wandb: 	number_neurons: 64
wandb: 	optimization_function: stochastic_gd
wandb: Currently logged in as: na20b064 (siddharth-s). Use `wandb login --relogin` to force relogin


100%|██████████| 5/5 [02:34<00:00, 30.94s/it]
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Training Accuracy: 10.05
Validation Accuracy: 9.55


Train_Accuracy,▁▁▁██
Train_Loss,█▃▁▁▁
Validation_Accuracy,███▁▁
Validation_loss,█▄▂▁▁
epoch,▁▃▅▆█
Train_Accuracy,10.05
Train_Loss,2.30258
Validation_Accuracy,9.55
Validation_loss,2.30274
epoch,4


wandb: Agent Starting Run: w5oc79hw with config:
wandb: 	activation_function: sigmoid
wandb: 	initialization_type: xavier
wandb: 	lambd: 0.5
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	mini_batch_size: 16
wandb: 	number_hidden_layers: 3
wandb: 	number_neurons: 64
wandb: 	optimization_function: nesterovacc_gd


Wrong optimization function


Run w5oc79hw errored: UnboundLocalError("local variable 'optimization_function' referenced before assignment")
wandb: ERROR Run w5oc79hw errored: UnboundLocalError("local variable 'optimization_function' referenced before assignment")
wandb: Agent Starting Run: km2rl5te with config:
wandb: 	activation_function: relu
wandb: 	initialization_type: random
wandb: 	lambd: 0.0005
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	mini_batch_size: 16
wandb: 	number_hidden_layers: 3
wandb: 	number_neurons: 128
wandb: 	optimization_function: stochastic_gd


  0%|          | 0/5 [00:00<?, ?it/s]wandb: Ctrl + C detected. Stopping sweep.
